# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fe5ecd1a490>
├── 'a' --> tensor([[-0.1905,  0.0698,  1.1688],
│                   [-1.7544,  0.5436,  0.7686]])
└── 'x' --> <FastTreeValue 0x7fe5ecd1ac40>
    └── 'c' --> tensor([[-0.6593,  0.6122, -0.0992, -0.9266],
                        [-0.8774, -0.1818,  1.8838,  1.4354],
                        [-1.6157, -1.2327,  0.7128, -0.1934]])

In [4]:
t.a

tensor([[-0.1905,  0.0698,  1.1688],
        [-1.7544,  0.5436,  0.7686]])

In [5]:
%timeit t.a

64.3 ns ± 0.00756 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fe5ecd1a490>
├── 'a' --> tensor([[ 0.8981,  0.2205,  0.7297],
│                   [ 0.2725, -0.4632, -0.1100]])
└── 'x' --> <FastTreeValue 0x7fe5ecd1ac40>
    └── 'c' --> tensor([[-0.6593,  0.6122, -0.0992, -0.9266],
                        [-0.8774, -0.1818,  1.8838,  1.4354],
                        [-1.6157, -1.2327,  0.7128, -0.1934]])

In [7]:
%timeit t.a = new_value

62.1 ns ± 0.045 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.1905,  0.0698,  1.1688],
               [-1.7544,  0.5436,  0.7686]]),
    x: Batch(
           c: tensor([[-0.6593,  0.6122, -0.0992, -0.9266],
                      [-0.8774, -0.1818,  1.8838,  1.4354],
                      [-1.6157, -1.2327,  0.7128, -0.1934]]),
       ),
)

In [10]:
b.a

tensor([[-0.1905,  0.0698,  1.1688],
        [-1.7544,  0.5436,  0.7686]])

In [11]:
%timeit b.a

57.7 ns ± 0.0191 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.5597,  0.2333, -0.6854],
               [ 0.6173,  0.3852,  0.0096]]),
    x: Batch(
           c: tensor([[-0.6593,  0.6122, -0.0992, -0.9266],
                      [-0.8774, -0.1818,  1.8838,  1.4354],
                      [-1.6157, -1.2327,  0.7128, -0.1934]]),
       ),
)

In [13]:
%timeit b.a = new_value

485 ns ± 0.11 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

858 ns ± 19.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 24.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

135 µs ± 304 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

141 µs ± 934 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fe535fd3370>
├── 'a' --> tensor([[[-0.1905,  0.0698,  1.1688],
│                    [-1.7544,  0.5436,  0.7686]],
│           
│                   [[-0.1905,  0.0698,  1.1688],
│                    [-1.7544,  0.5436,  0.7686]],
│           
│                   [[-0.1905,  0.0698,  1.1688],
│                    [-1.7544,  0.5436,  0.7686]],
│           
│                   [[-0.1905,  0.0698,  1.1688],
│                    [-1.7544,  0.5436,  0.7686]],
│           
│                   [[-0.1905,  0.0698,  1.1688],
│                    [-1.7544,  0.5436,  0.7686]],
│           
│                   [[-0.1905,  0.0698,  1.1688],
│                    [-1.7544,  0.5436,  0.7686]],
│           
│                   [[-0.1905,  0.0698,  1.1688],
│                    [-1.7544,  0.5436,  0.7686]],
│           
│                   [[-0.1905,  0.0698,  1.1688],
│                    [-1.7544,  0.5436,  0.7686]]])
└── 'x' --> <FastTreeValue 0x7fe535fd3d90>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.4 µs ± 38.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fe535fd32b0>
├── 'a' --> tensor([[-0.1905,  0.0698,  1.1688],
│                   [-1.7544,  0.5436,  0.7686],
│                   [-0.1905,  0.0698,  1.1688],
│                   [-1.7544,  0.5436,  0.7686],
│                   [-0.1905,  0.0698,  1.1688],
│                   [-1.7544,  0.5436,  0.7686],
│                   [-0.1905,  0.0698,  1.1688],
│                   [-1.7544,  0.5436,  0.7686],
│                   [-0.1905,  0.0698,  1.1688],
│                   [-1.7544,  0.5436,  0.7686],
│                   [-0.1905,  0.0698,  1.1688],
│                   [-1.7544,  0.5436,  0.7686],
│                   [-0.1905,  0.0698,  1.1688],
│                   [-1.7544,  0.5436,  0.7686],
│                   [-0.1905,  0.0698,  1.1688],
│                   [-1.7544,  0.5436,  0.7686]])
└── 'x' --> <FastTreeValue 0x7fe535fd3a00>
    └── 'c' --> tensor([[-0.6593,  0.6122, -0.0992, -0.9266],
                        [-0.8774, -0.1818,  1.8838,  1.4354],
                 

In [23]:
%timeit t_cat(trees)

29.8 µs ± 58.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

60 µs ± 93.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.6593,  0.6122, -0.0992, -0.9266],
                       [-0.8774, -0.1818,  1.8838,  1.4354],
                       [-1.6157, -1.2327,  0.7128, -0.1934]],
              
                      [[-0.6593,  0.6122, -0.0992, -0.9266],
                       [-0.8774, -0.1818,  1.8838,  1.4354],
                       [-1.6157, -1.2327,  0.7128, -0.1934]],
              
                      [[-0.6593,  0.6122, -0.0992, -0.9266],
                       [-0.8774, -0.1818,  1.8838,  1.4354],
                       [-1.6157, -1.2327,  0.7128, -0.1934]],
              
                      [[-0.6593,  0.6122, -0.0992, -0.9266],
                       [-0.8774, -0.1818,  1.8838,  1.4354],
                       [-1.6157, -1.2327,  0.7128, -0.1934]],
              
                      [[-0.6593,  0.6122, -0.0992, -0.9266],
                       [-0.8774, -0.1818,  1.8838,  1.4354],
                       [-1.6157, -1.2327,  0.7128, -0.1934]],

In [26]:
%timeit Batch.stack(batches)

78.2 µs ± 102 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.6593,  0.6122, -0.0992, -0.9266],
                      [-0.8774, -0.1818,  1.8838,  1.4354],
                      [-1.6157, -1.2327,  0.7128, -0.1934],
                      [-0.6593,  0.6122, -0.0992, -0.9266],
                      [-0.8774, -0.1818,  1.8838,  1.4354],
                      [-1.6157, -1.2327,  0.7128, -0.1934],
                      [-0.6593,  0.6122, -0.0992, -0.9266],
                      [-0.8774, -0.1818,  1.8838,  1.4354],
                      [-1.6157, -1.2327,  0.7128, -0.1934],
                      [-0.6593,  0.6122, -0.0992, -0.9266],
                      [-0.8774, -0.1818,  1.8838,  1.4354],
                      [-1.6157, -1.2327,  0.7128, -0.1934],
                      [-0.6593,  0.6122, -0.0992, -0.9266],
                      [-0.8774, -0.1818,  1.8838,  1.4354],
                      [-1.6157, -1.2327,  0.7128, -0.1934],
                      [-0.6593,  0.6122, -0.0992, -0.9266],
                   

In [28]:
%timeit Batch.cat(batches)

143 µs ± 149 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

307 µs ± 355 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
